In [27]:
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from keras.layers import  Dropout, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

In [28]:
# load in the data
df = pd.read_csv('C:/Users/Jeferson Santana/OneDrive/Documentos/recommender_course/data/edited_rating.csv')

In [29]:
N = df.userId.max() + 1 # number of users
M = df.movie_idx.max() + 1 # number of movies

In [30]:
# split into train and test
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

In [31]:
# initialize variables
K = 10 # latent dimensionality
mu = df_train.rating.mean()
epochs = 25

In [32]:
# keras model
u = Input(shape=(1,))
m = Input(shape=(1,))
u_embedding = Embedding(N, K)(u)
m_embedding = Embedding(M, K)(m)
u_embedding = Flatten()(u_embedding)
m_embedding = Flatten()(m_embedding)
x = Concatenate()([u_embedding, m_embedding])

In [33]:
# the neural network
x = Dense(400)(x)
x = Activation('relu')(x)
x = Dense(1)(x)

In [ ]:
model = Model(inputs=[u ,m], outputs=x)
model.compile(
                loss='mse',
                optimizer=SGD(learning_rate=0.01, momentum=0.9),
                metrics=['mse']
             )

r = model.fit(
                x=[df_train.userId.values, df_train.movie_idx.values],
                y=df_train.rating.values - mu,
                epochs=epochs,
                batch_size=128,
                validation_data=(
                                    [df_test.userId.values, df_test.movie_idx.values],
                                    df_test.rating.values - mu
                                )
              )

In [ ]:
# plot losses
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="test loss")
plt.legend()
plt.show()

# plot mse
plt.plot(r.history['mean_squared_error'], label="train mse")
plt.plot(r.history['val_mean_squared_error'], label="test mse")
plt.legend()
plt.show()